In [8]:
import pandas as pd
import numpy as np
import sqlite3


def age_group(x):
    if x <= 20:
        return 1
    if x > 20 and x<=30:
        return 2
    if x > 30 and x<=40:
        return 3
    if x > 40 and x<=50:
        return 4
    if x > 50 and x<=60:
        return 5
    if x > 60:
        return 6
    return 0



df=pd.read_csv("clientes.csv",sep=';',na_filter=True)
df["correo"].fillna("s/c",inplace=True)
df["estatus_contacto"].fillna("s/e",inplace=True)
df["prioridad"]=df["prioridad"].fillna(0).astype(int)
df[["fiscal_id","first_name","last_name","gender","direccion","correo", "estatus_contacto"]]=df[["fiscal_id","first_name","last_name","gender","direccion","correo", "estatus_contacto"]].apply(lambda x: x.str.upper())
df["telefono"]=df['telefono'].fillna(0.0).astype(str).str.rsplit(".",expand=True).replace("0","s/n")
#df.to_excel (r'export_dataframe3.xlsx', index = False, header=True)
print(df.dtypes)


fiscal_id            object
first_name           object
last_name            object
gender               object
fecha_nacimiento     object
fecha_vencimiento    object
deuda                 int64
direccion            object
altura                int64
peso                  int64
correo               object
estatus_contacto     object
prioridad             int32
telefono             object
dtype: object


In [10]:
clientes=df[["fiscal_id","first_name","last_name","gender","fecha_nacimiento","fecha_vencimiento","deuda","direccion"]]
clientes=clientes.rename(columns={ 'fecha_nacimiento':'birth_date','fecha_vencimiento':'due_date','deuda':'due_balance','direccion':'address'})
clientes['birth_date']=pd.to_datetime(clientes['birth_date'])
clientes['due_date']=pd.to_datetime(clientes['due_date'])
clientes['age'] =pd.to_datetime("today").year-clientes['birth_date'].apply(lambda x: x.year)
clientes['delinquency'] =pd.to_datetime("today")-clientes['due_date']
clientes['delinquency']=clientes['delinquency'].dt.days
#clientes.dtypes

In [11]:
#clientes.apply(age_group, axis='columns')


In [12]:
clientes["age_group"]=clientes["age"].apply(lambda x: age_group(x))
clientes = clientes.reindex(columns=['fiscal_id',
                                     'first_name',
                                     'last_name',
                                     'gender',
                                     'birth_date',
                                     'age',
                                     'age_group',
                                     'due_date',
                                     'delinquency',
                                     'due_balance',
                                     'address'
                                    ])
clientes['birth_date']=clientes['birth_date'].dt.date
clientes['due_date']=clientes['due_date'].dt.date

In [13]:

clientes.to_excel (r'export_dataframe.xlsx', index = False, header=True)
clientes

,fiscal_id,first_name,last_name,gender,birth_date,age,age_group,due_date,delinquency,due_balance,address
0,41631392-K,WENDY,WARE,MALE,1990-11-11,31,3,2021-04-23,160,658,8679 PRIMIS AVENUE
1,49665408-0,IMELDA,CAMACHO,MALE,1995-05-10,26,2,2021-01-29,244,844,252-8718 DICTUM RD.
2,24247506-2,ERICA,DELANEY,FEMALE,1996-09-09,25,2,2021-02-02,240,297,AP #198-2498 FELIS ROAD
3,34653654-3,AUDRA,PACE,FEMALE,1993-11-02,28,2,2021-01-13,260,376,4481 DONEC AV.
4,39455789-7,NEHRU,FISHER,MALE,1994-01-24,27,2,2021-01-07,266,966,"461-7845 DUI, STREET"
...,...,...,...,...,...,...,...,...,...,...,...
595,48121035-6,MARA,POTTER,FEMALE,1998-08-22,23,2,2021-02-04,238,687,178-3433 MI STREET
596,32761397-9,COLORADO,ROSALES,MALE,1996-06-13,25,2,2021-04-25,158,323,"160-9900 NON, RD."
597,43471829-5,JACOB,KELLER,MALE,1991-02-18,30,2,2021-03-27,187,755,2196 DOLOR. AVE
598,42169978-K,SHEILA,HOOPER,FEMALE,1996-12-28,25,2,2021-04-12,171,998,AP #822-6862 NISL RD.


In [14]:
emails=df[["fiscal_id",'correo','estatus_contacto','prioridad']]
emails=emails.rename(columns={'correo':'email','estatus_contacto':'status','prioridad':'priority'}) 
emails.head()
emails.dtypes

fiscal_id    object
email        object
status       object
priority      int32
dtype: object

In [15]:
phone=df[["fiscal_id",'telefono','estatus_contacto','prioridad']]
phone=phone.rename(columns={'telefono':'phone','estatus_contacto':'status','prioridad':'priority'})
phone.head()
phone.dtypes

fiscal_id    object
phone        object
status       object
priority      int32
dtype: object

In [19]:
bd = sqlite3.connect('database.db3')
df.to_sql(name='df', con=bd)
clientes.to_sql(name='clientes1', con=bd)
emails.to_sql(name='emails', con=bd)
phone.to_sql(name='phone', con=bd)

In [20]:
bd.close()